In [52]:
import hashlib
import re
from datetime import datetime, timezone
import pandas as pd
import sqlite3

In [53]:
EXCLUSION_LIST = ['BLANK', '-', 'NA', 'NONE', '{NULL}', 'VIDE']

In [54]:
# clean column names
def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
    return df

In [55]:
def clean_dob(dob, age):
    if dob > datetime.now() and age < 100:
        dob = dob.replace(year=datetime.now().year - age)
    if age >= 100 and dob.year > datetime.now().year - 100:
        dob = dob.replace(year=datetime.now().year - age)
    return dob

In [56]:
def clean_column(value, exclusions=EXCLUSION_LIST):
    if isinstance(value, str) and value.strip().upper() in exclusions:
        return None
    elif isinstance(value, float) and pd.isna(value):
        return None
    else:
        return value

In [57]:
def hash_password(pw: str) -> str:
    if pw is None:
        return None
    else:
        return hashlib.sha256(pw.encode('utf-8')).hexdigest()

In [58]:
def clean_salary(salary: str, period:int=1) -> float:
    if salary is None:
        return None
    else:
        salary = (int(re.sub(r'[^\d]', '', salary))/100) * period
        return salary

In [59]:
def clean_phone_number(df:pd.DataFrame, column:str) -> pd.DataFrame:
    df[column] = df[column].str.replace(' ', '')
    return df

In [60]:
def load_user_data(filepath:str,
                   encoding:str='utf-8',
                   date_format:str='%d/%m/%y',
                   mapping=None):
    df = pd.read_csv(filepath, encoding=encoding)
    if mapping:
        df.rename(columns=mapping, inplace=True)
    df = clean_cols(df)
    df['dob'] = pd.to_datetime(df['dob'], format=date_format, errors='coerce')
    return df

In [61]:
def transform_users_df(users:pd.DataFrame,
                       country_code:str,
                       currency:str='GBP',
                       salary_period:int=1,
                       mapping_gender=None,
                       mapping_education=None):
    for col in users.columns:
        users[col] = users[col].apply(lambda x: clean_column(x, EXCLUSION_LIST))
    # hash passwords
    users['password'] = users['password'].apply(hash_password)
    # clean dob
    users['dob'] = users.apply(lambda x: clean_dob(x['dob'], x['age_last_birthday']), axis=1)
    # clean gender
    if mapping_gender:
        users['gender'] = users['gender'].replace(mapping_gender)
    # clean phone numbers
    clean_phone_number(users, 'phone')
    clean_phone_number(users, 'mobile')
    # clean salary
    users['salary'] = users['salary'].apply(lambda x: clean_salary(x, salary_period))
    users['currency'] = currency
    # add rqf column if it doesn't exist
    if mapping_education and 'rqf' not in users.columns:
        users['rqf'] = users['education'].map(mapping_education)
    # add country code
    users['country_code'] = country_code
    return users

In [62]:
fr_mapping_columns = {
    'Prénom': 'first_name',
    'Nom de famille': 'surname',
    'DdN': 'dob',
    '\nÂge dernier anniversaire': 'age_last_birthday',
    'Couleur préférée': 'favourite_colour',
    'Animal préféré': 'favourite_animal',
    'Plat préféré': 'favourite_food',
    'Genre': 'gender',
    'Mot de passe': 'password',
    'Ville': 'city',
    'Département': 'county',
    'Code postal': 'postcode',
    'Adresse électronique': 'email',
    'Téléphone': 'phone',
    'Portable ': 'mobile',
    'BAC+': 'education',
    ' Salaire ': 'salary',
    'Visites du site Web au cours des 30 derniers jours': 'website_visits_last_30_days'
 }

In [74]:
fr_mapping_gender = {'F':'Female', 'M': 'Male', 'NB': 'Non-Binary'}
usa_mapping_gender = {'0':'Female', '1': 'Male'}

In [77]:
fr_mapping_education = {
    'Baccalauréat':3,
    'Licentiate':6,
    'Master':7,
    'CFA':5,
    'Collège':1,
    'Lycée':2,
    'Doctorat':8
    }
usa_mapping_education = {
    'High School Diploma':3,
    'Associate Degree':4,
    'Foundation Degree':5,
    'Bachelor Degree':6,
    "Master’s":7,
    'Doctorate':8
} 


In [ ]:
# users_usa['education'].map(usa_mapping_education)

0    3
1    4
2    3
3    3
4    5
5    6
6    6
7    7
8    8
9    5
Name: education, dtype: int64

In [65]:
users_uk = load_user_data('data/UK User Data.csv',
                          encoding='latin1')
users_uk = transform_users_df(users_uk,
                              country_code='UK')
users_fr = load_user_data('data/FR User Data.csv',
                          date_format='%y-%m-%d',
                          mapping=fr_mapping_columns)
users_fr = transform_users_df(users_fr,
                              country_code='FR',
                              currency='EUR',
                              salary_period=12,
                              mapping_gender=fr_mapping_gender,
                              mapping_education=fr_mapping_education)

In [71]:
usa_mapping_columns = {
    'Last Name':'surname',
    'Favorite Color':'favourite_colour',
    'Favorite Animal':'favourite_animal',
    'Favorite Food':'favourite_food',
    'Town/City':'city',
    'Zip Code':'postcode',
    'Landline':'phone',
    'Cell Phone':'mobile'
} 

In [72]:
users_usa = load_user_data('data/USA User Data.csv',
                          date_format='%m/%d/%y',
                          mapping=usa_mapping_columns) 
users_usa.head(10)

,first_name,surname,middle_initials,dob,age_last_birthday,favourite_colour,favourite_animal,favourite_food,gender,password,city,state,state_code,postcode,email,phone,mobile,education,salary,website_visits_last_30_days
0,Michael,Smith,J,2045-06-05,79,Gray,Moose,Apple Pie,1,Pa$$w0rdMS,Palmer,Alaska,AK,99645,smithmj45@gmail.com,907-775-6622,907-319-6703,High School Diploma,"$27,000.00",16
1,Hannah,Johnson,NaN,2001-11-23,23,Pink,Bighorn Sheep,Buffalo Wings,0,Pa$$w0rdHJ,Granby,Colorado,CO,80446,johnsonm01@outlook.com,970-524-5555,970-844-3539,Associate Degree,"$78,260.80",12
2,Jacob,Brown,M,1975-01-25,50,Blue,Fox,Cheeseburger,1,Pa$$w0rdJB,Kenton,Delaware,DE,19955,brownmm75@yahoo.com,302-399-1876,302-278-1947,High School Diploma,"$18,620.00",7
3,Ashley,Williams,NaN,2007-11-19,17,Purple,Panther,Clam Chowder,0,Pa$$w0rdAW,Andover,Florida,FL,33179,williamsm07@hotmail.com,(305)654-2727,305-309-1054,High School Diploma,"$22,080.00",28
4,Joshua,Wong,M,1985-08-08,39,Yellow,Seal,Mac & Cheese,1,Pa$$w0rdJW,Whitmore Village,Hawaii,HI,96786,wongmm85@protonmail.com,(808) 307-3500,(808) 622-6340,Foundation Degree,"$82,560.00",19
5,Emily,Miller,S,1981-07-12,43,Green,Muskrat,Pizza,0,Pa$$w0rdEM,Cedar Rapids,Iowa,IA,52227,millerms81@mail.com,319-471-3533,319-363-8294,Bachelor Degree,"$117,600.00",34
6,Tyler,Nelson,NaN,1993-01-20,32,Navy,Deer,Hot Dogs,1,Pa$$w0rdTN,Andover,Maine,ME,4216,nelsonm93@proton.me,(207) 475-0404,207-475-8295,Bachelor Degree,"$98,000.00",27
7,Sarah,Garcia,T,2001-06-07,23,Ebony,Bobcat,Grilled Cheese,0,Pa$$w0rdSG,Andover,New Hampshire,NH,3216,garciamt01@gmail.com,(603) 528-3488,603-267-3849,Master’s,"$136,515.00",63
8,Austin,Anderson,NaN,2064-06-30,60,Mauve,Beaver,Fried Chicken,1,Pa$$w0rdAA,Salem,Oregon,OR,97301,andersonm64@yahoo.com,5035887272,503‑588‑6245,Doctorate,"$170,900.00",23
9,Madison,Olson,O,2044-04-10,80,Hazel,Elk,Biscuits 'n' Gravy,0,Pa$$w0rdMO,Leeds,Utah,UT,84746,olsonmo44@gmail.com,435-628-8320,435-267-4866,Foundation Degree,"$78,000.00",36


In [68]:
def load_login_data(filepath:str,timezone:str='Europe/London'):
    df = pd.read_csv(filepath)
    df.columns = ['login_id', 'username', 'login_timestamp']
    df.drop(columns=['login_id'], inplace=True)
    df['login_timestamp'] = pd.to_datetime(df['login_timestamp'], unit='s', utc=False)
    df['login_timestamp'] = df['login_timestamp'].dt.tz_localize(timezone).dt.tz_convert('UTC')
    return df

In [69]:
logins_uk = load_login_data('data/UK-User-LoginTS.csv')
logins_fr = load_login_data('data/FR-User-LoginTS.csv', timezone='Europe/Paris')

In [70]:
with open('create_database.sql', 'r', encoding='utf-8') as f:
    create_sql = f.read()
conn = sqlite3.connect('customers.db')
try:
    conn.executescript(create_sql)
    users_uk.to_sql('users', conn, if_exists='append', index=False)
    users_fr.to_sql('users', conn, if_exists='append', index=False)
    logins_uk.to_sql('logins', conn, if_exists='append', index=False)
    logins_fr.to_sql('logins', conn, if_exists='append', index=False)
    conn.commit()
finally:
    conn.close()